# --- Day 8: Seven Segment Search ---
https://adventofcode.com/2021/day/8

Different digits have different number of segments:
2 segments: 1
3 segments: 7
4 segments: 4
5 segments: 2,3,5
6 segments: 0,6,9
7 segments: 8

In [10]:
import aocd
data = aocd.get_data(day=8, year=2021).splitlines()
# # # # Test data
# with open('input\Day 08.txt', 'r') as f:
#     data = f.read().splitlines()

In [6]:
def parse_data(data):
    '''
    For each list element, returns a list, containing 
    a list of 10 digits (all digits) and a list of 4 digits (value)
    '''
    res = []
    for line in data:
        [part1, part2] = line.split('|')
        digits = part1.split()
        output = part2.split()
        res.append([digits,output])
    return res

def count_1478(output):
    '''
    Returns total number of digits 1,4,7, and 8 in the given output,
    i.e. in a list of 4 values
    '''
    return sum([len(x) in [2,3,4,7] for x in output])

def part1(data):
    data_parsed = parse_data(data)
    return sum([count_1478(line[1]) for line in data_parsed])
    

In [7]:
answer1 = part1(data)
print(f'Part 1: {answer1}')

Part 1: 26


Part 2. Given the segments oriented like this:

 aaaa  
b    c
b    c
 dddd 
e    f
e    f
 gggg 
 
it can be shown that:

1. 'a' is in len==4 but not in len==2
2. 'd' is in len==4 and all len==5
3. 'g' is the remaining one in all len==5
4. 'b' is the remaining one in len==4 which is not in len==2
5. 'f' is the remaining one in all len==6
6. 'c' is the remaining one in len==2
7. 'e' is the remaining one

Replacement rule will be a dict like: {'a':'c', ...}. Here 'a' is the value in data (to which segment the wire is connected), and 'c' is the expected value (to what segment the wire should be connected). It is easier to decode in this case.

In [13]:
from collections import Counter

def sort(string):
    '''
    Sort the letters in a string in alphabetical order
    '''
    return ''.join(sorted(string))

def find_rule(digits):
    '''
    Processes a list of 10 strings (digits) and returns a dict 
    for replacement of segments
    '''
    rule = {}
    remaining = 'abcdefg'
    lens = [len(x) for x in digits]

    # digits and groups of digits
    n1 = set(digits[lens.index(2)])
    n7 = digits[lens.index(3)]
    n4 = digits[lens.index(4)]
    n235 = [digits[i] for i, x in enumerate(lens) if x==5]
    n069 = [digits[i] for i, x in enumerate(lens) if x==6]
    
    # 1. 'a' is in len==4 but not in len==1
    segm_a = list(set(n7) ^ set(n1))[0]
    rule[segm_a] = 'a'
    remaining = remaining.replace(segm_a,'')
    
    # 2. 'd' is in len==4 and all len==5
    segm_d = list(set(n4) & set(n235[0]) & set(n235[1]) & set(n235[2]))[0]
    rule[segm_d] = 'd'
    remaining = remaining.replace(segm_d,'')
    
    # 3. 'g' is the remaining one in all len==5
    segm_g = list(set(remaining) & set(n235[0]) & set(n235[1]) & set(n235[2]))[0]
    rule[segm_g] = 'g'
    remaining = remaining.replace(segm_g,'')
    
    # 4. 'b' is the remaining one in len==4 which is not in len==2
    segm_b = list(set(remaining) & (set(n4)^set(n1)))[0]
    rule[segm_b] = 'b'
    remaining = remaining.replace(segm_b,'')  
    
    # 5. 'f' is the remaining one in all len==6
    segm_g = list(set(remaining) & set(n069[0]) & set(n069[1]) & set(n069[2]))[0]
    rule[segm_g] = 'f'
    remaining = remaining.replace(segm_g,'')
    
    # 'c' is the remaining one in len==2
    segm_c = list(set(remaining) & set(n1))[0]
    rule[segm_c] = 'c'
    remaining = remaining.replace(segm_c,'')
    
    # 7. 'e' is the remaining one
    segm_e = remaining
    rule[segm_e] = 'e'
    
    return rule

def decode(output, rule):
    '''
    Decodes the output data (list of strings) according to the rule (a dict)
    '''
    decoded_output = []
    for line in output:
        decoded_line = ''.join([rule[x] for x in line])
        decoded_output.append(decoded_line)
    return decoded_output   

def to_number(output):
    '''
    Translates decoded output (list of 4 digits) to a 4-digit number
    '''
    digits = ['abcefg','cf','acdeg','acdfg','bcdf', 
              'abdfg','abdefg','acf','abcdefg','abcdfg']
    number = 0
    for i,string in enumerate(output):
        dgt = digits.index(sort(string))
        number += dgt*10**(3-i) 
    return number



def part2(data):
    data_parsed = parse_data(data)
    summ = 0;
    for line in data_parsed:
        rule = find_rule(line[0])
        decoded = decode(line[1],rule)
        out = to_number(decoded)
        summ += out
    return summ

In [14]:
answer2 = part2(data)
print(f'Part 2: {answer2}')

Part 2: 1016804
